# GRU 网络

In [14]:
import torch
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
from imp import reload

import sys

sys.path.append("..")
import d2l_pytorch.d2l as d2l

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

reload(d2l)


<module 'd2l_pytorch.d2l' from '..\\d2l_pytorch\\d2l.py'>

In [3]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics("F:\python_code\DL\Datasets\JayChouLyrics\jaychou_lyrics.txt")

In [9]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print("will use", device)

def get_params():
  def _one(shape):
    ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype = torch.float32)
    return torch.nn.Parameter(ts, requires_grad=True)
  def _three():
    return (_one((num_inputs, num_hiddens)), _one((num_hiddens, num_hiddens)), torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
  
  W_xz, W_hz, b_z = _three()
  W_xr, W_hr, b_r = _three()
  W_xh, W_hh, b_h = _three()

  W_hq = _one((num_hiddens, num_outputs))
  b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32),requires_grad=True)
  return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

will use cuda


In [6]:
def init_gru_state(batch_size, num_hiddens, device):
  return (torch.zeros((batch_size, num_hiddens), device=device),)

In [7]:
def gru(inputs, state, params):
  W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
  (H,) = state
  outputs = []

  for X in inputs:
    Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
    R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
    H_tilda = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(R * H, W_hh) + b_h)
    H = Z * H + (1 - Z) * H_tilda
    Y = torch.matmul(H, W_hq) + b_q
    outputs.append(Y)
  return outputs, (H,)


In [10]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ["分开", "不分开"]

d2l.train_and_predict_rnn(
  gru,
  get_params,
  init_gru_state,
  num_hiddens,
  vocab_size,
  device,
  corpus_indices,
  idx_to_char,
  char_to_idx,
  False,
  num_epochs,
  num_steps,
  lr,
  clipping_theta,
  batch_size,
  pred_period,
  pred_len,
  prefixes,
)


epoch 40, perplexity 143.895503, time 0.92 sec
 - 分开
我想你你 我不了
我想你你你
我想你你不了
我想你你你
我想你你不了
我想你你你
我想你你不了
我
 - 不分开
我想你你 我不了
我想你你你
我想你你不了
我想你你你
我想你你不了
我想你你你
我想你你不了
我
epoch 80, perplexity 27.395228, time 0.96 sec
 - 分开
我想要这样
我该好好生活
我该好好生活
我该好好生活
我该好好生活
我该好好生活
我该好好生活
我
 - 不分开
我想要这样
我该好好生活
我该好好生活
我该好好生活
我该好好生活
我该好好生活
我该好好生活
我
epoch 120, perplexity 4.958879, time 0.92 sec
 - 分开的模丽
让我说你的爱写在西元前
深埋在美索不达米亚平原
用楔形文字我妈下难发
我爱上的话写有西元前

 - 不分开
爱是不觉 你来一碗热留
你这在黑秋 静静悔够不留
景色入烛 温暖了空屋
白色蜡烛 温暖了空屋
白色
epoch 160, perplexity 1.653393, time 1.08 sec
 - 分开
让我心到你
就样去一个人
后知回觉
后知后觉不离
我该好好生活
静静悄悄默默离开
陷入了危险边缘B
 - 不分开
其是我怕开痛痛
像像样童话 我有没直 
没有你烦我有多烦熬多
爸没有你的我有多
家散 没爱简久了吧


In [15]:
# 简洁实现使用nn中的GRU类
lr = 1e-2
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size).to(device)

d2l.train_and_predict_rnn_pytorch(
  model,
  num_hiddens,
  vocab_size,
  device,
  corpus_indices,
  idx_to_char,
  char_to_idx,
  num_epochs,
  num_steps,
  lr,
  clipping_theta,
  batch_size,
  pred_period,
  pred_len,
  prefixes,
)


 epoch 40, perplexity 1.031442 , time 0.13 sec 
 - 分开始打呼
管家是一只会说法语举止优雅的猪
吸血前会念约翰福音做为弥补
拥有一双蓝色眼睛的凯萨琳公主
专
 - 不分开不了口让她知道
就是那么简单几句 我办不到
整颗心悬在半空
我只能够远远看著
这些我都做得到
但那个
 epoch 80, perplexity 1.045292 , time 0.13 sec 
 - 分开的玩笑
想通 却又再考倒我
说散 你想很久了吧?
败给你的黑色幽默
说散 你想很久了吧?
我的认真败
 - 不分开暴风圈来不及逃
我不能再想
我不能再想
我不 我不 我不能
爱情走的太快就像龙卷风
离不开暴风圈来不
 epoch 120, perplexity 1.014025 , time 0.14 sec 
 - 分开的玩笑
想通 却又再考倒我
说散 你想很久了吧?
败给你的黑色幽默
说散 你想很久了吧?
我的认真败
 - 不分开不了口让她知道
就是那么简单几句 我办不到
整颗心悬在半空
我只能够远远看著
这些我都做得到
但那个
 epoch 160, perplexity 1.535654 , time 0.13 sec 
 - 分开现迷了路上
你的脑袋有问题
随便说说
是那句抱歉 在感动 谁让它比谁下午
手牵手
一步两步三步四步望
 - 不分开我爱你想想你想想你想很久了吧?
想通 却又再考倒我
说散 你想很久了吧?
败给黑色幽默
走过了很多地
